Import movies_metadata.csv.Picking out the title,overview,genres and movie id from the original csv.
Some genres seem to occur only once in the entire dataset. Hence, we remove those. In addition to this, genres like "TV Movie" and "Foreign" are not easily percieved from posters and movie summaries. Hence, those were removed as well.

In order to avoid repeated downloading of posters,we use pkl file to save the poster_paths.

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import ast

style.use('seaborn-poster')
style.use('ggplot')

data = pd.read_csv("content/movies_metadata.csv",low_memory=False)

filtered_data = data.filter(['original_title','overview', 'id', 'genres'], axis=1)

genres = filtered_data['genres'].tolist()

genre_per_movie = []
genre_dict = {}
invalid_genres = ['Aniplex', 'BROSTA TV', 'Carousel Productions', 'GoHands',
                  'Mardock Scramble Production Committee', 'Odyssey Media',
                  'Pulser Productions', 'Rogue State', 'Sentai Filmworks',
                  'Telescene Film Group Productions', 'The Cartel', 'Vision View Entertainment', 
                  'TV Movie', 'Foreign']
for genre in genres:
    genre = ast.literal_eval(genre)
    genre_list = []
    for val in genre:
        if val['name'] not in invalid_genres:
            genre_list.append(val['name'])
            if val['name'] in genre_dict:
                genre_dict[val['name']] += 1
            else:
                genre_dict[val['name']] = 1
    genre_per_movie.append(genre_list)

genre_dict = {key:val for key, val in genre_dict.items() if val != 1}

filtered_data['genre_list'] = genre_per_movie

filtered_data = filtered_data[filtered_data['genre_list'].map(lambda d: len(d)) > 0]

total_empty = 0
for genre in genre_per_movie:
  if not len(genre):
    total_empty += 1


import joblib
poster_paths = joblib.load('poster_path1.pkl')


filtered_data['poster_paths'] = poster_paths

filtered_data = filtered_data[filtered_data['poster_paths'] != "API Error"]

filtered_data = filtered_data[filtered_data['poster_paths'] != "Error"]

In [63]:
poster_paths

['content/downloaded_posters/Toy Story.jpg',
 'content/downloaded_posters/Jumanji.jpg',
 'content/downloaded_posters/Grumpier Old Men.jpg',
 'content/downloaded_posters/Waiting to Exhale.jpg',
 'content/downloaded_posters/Father of the Bride Part II.jpg',
 'content/downloaded_posters/Heat.jpg',
 'content/downloaded_posters/Sabrina.jpg',
 'content/downloaded_posters/Tom and Huck.jpg',
 'content/downloaded_posters/Sudden Death.jpg',
 'content/downloaded_posters/GoldenEye.jpg',
 'content/downloaded_posters/The American President.jpg',
 'content/downloaded_posters/Dracula: Dead and Loving It.jpg',
 'content/downloaded_posters/Balto.jpg',
 'content/downloaded_posters/Nixon.jpg',
 'content/downloaded_posters/Cutthroat Island.jpg',
 'content/downloaded_posters/Casino.jpg',
 'content/downloaded_posters/Sense and Sensibility.jpg',
 'content/downloaded_posters/Four Rooms.jpg',
 'content/downloaded_posters/Ace Ventura: When Nature Calls.jpg',
 'content/downloaded_posters/Money Train.jpg',
 'conte

use csv file to save data

In [29]:
filtered_data.to_csv('content/movie_prediction/dataset_mod.csv')
filtered_data = pd.read_csv("content/movie_prediction/dataset_mod.csv")

splitting the data into training,testing and validation sets.

In [64]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(filtered_data, test_size=0.2)
train, val = train_test_split(train, test_size=0.1)


val.to_csv("content/movie_prediction/val_data.csv")
test.to_csv("content/movie_prediction/test_data.csv")
train.to_csv("content/movie_prediction/train_data.csv")

All downloaded images are read, resized and their pixels normalized between 0 and 1. Each image is then appended to a list. In case any images are not readable, those rows are dropped. The list of images obtained for the train, test and validation datasets are then saved as numpy arrays. This was done to easily load them later as an input to the model.

In [62]:
import cv2
import os

def resize_img(path):
    if not os.path.exists(path):
        return None
    try:
        img = cv2.imread(path)
        img = cv2.resize(img, (75, 115))
        img = img.astype(np.float32)/255
        img = img - np.array([0.485, 0.456, 0.406])
        img = img / np.array([0.229, 0.224, 0.225])
        return img
    except Exception as e:
        print(str(e))
        return None


val_imgs = []
i = 0
for index, row in val.iterrows():
    if i%100 == 0:
        print("Processing i:", i, str(row['poster_paths']))
    img = resize_img(str(row['poster_paths']))
    if img is not None:
        val_imgs.append(img)
    else:
        val = val.drop(index)
        print("Dropping row:", index, "Length:", len(val))
    i+=1

Processing i: 0 content/downloaded_posters/Hot Rods To Hell.jpg
Dropping row: 34333 Length: 2848
Dropping row: 18514 Length: 2847
Dropping row: 21503 Length: 2846
Dropping row: 25555 Length: 2845
Dropping row: 14229 Length: 2844
Dropping row: 21280 Length: 2843
Dropping row: 21178 Length: 2842
Dropping row: 8705 Length: 2841
Dropping row: 19539 Length: 2840
Dropping row: 31310 Length: 2839
Dropping row: 11779 Length: 2838
Dropping row: 23978 Length: 2837
Dropping row: 11390 Length: 2836
Dropping row: 7640 Length: 2835
Dropping row: 7433 Length: 2834
Dropping row: 19469 Length: 2833
Processing i: 100 content/downloaded_posters/Never Take Sweets from a Stranger.jpg
Dropping row: 9621 Length: 2832
Dropping row: 27488 Length: 2831
Dropping row: 1096 Length: 2830
Dropping row: 11158 Length: 2829
Dropping row: 26373 Length: 2828
Dropping row: 33790 Length: 2827
Dropping row: 18189 Length: 2826
Dropping row: 32268 Length: 2825
Dropping row: 12821 Length: 2824
Dropping row: 25312 Length: 2823


Dropping row: 32273 Length: 2632
Dropping row: 13958 Length: 2631
Dropping row: 21349 Length: 2630
Dropping row: 34723 Length: 2629
Dropping row: 14327 Length: 2628
Dropping row: 12985 Length: 2627
Dropping row: 18682 Length: 2626
Dropping row: 27009 Length: 2625
Dropping row: 17846 Length: 2624
Dropping row: 2265 Length: 2623
Processing i: 1700 content/downloaded_posters/Je vous salue, Marie.jpg
Dropping row: 25487 Length: 2622
Dropping row: 9256 Length: 2621
Dropping row: 6702 Length: 2620
Dropping row: 28413 Length: 2619
Dropping row: 6960 Length: 2618
Dropping row: 21445 Length: 2617
Dropping row: 12925 Length: 2616
Dropping row: 19489 Length: 2615
Dropping row: 9598 Length: 2614
Dropping row: 8324 Length: 2613
Dropping row: 8948 Length: 2612
Processing i: 1800 content/downloaded_posters/Ice Quake.jpg
Dropping row: 29710 Length: 2611
Dropping row: 2831 Length: 2610
Dropping row: 20769 Length: 2609
Dropping row: 6558 Length: 2608
Dropping row: 35140 Length: 2607
Dropping row: 2157 L

In [16]:
assert len(val) == len(val_imgs)
val_np_imgs = np.array(val_imgs)
val.to_csv('content/movie_prediction/val_data.csv')

import pickle
with open('content/movie_prediction/val_np_imgs_norm','wb') as f: pickle.dump(val_np_imgs, f)


test_imgs = []
i = 0
for index, row in test.iterrows():
    if i%100 == 0:
        print("Processing i:", i, str(row['poster_paths']))
        img = resize_img(str(row['poster_paths']))
    if img is not None:
        test_imgs.append(img)
    else:
        test = test.drop(index)
        print("Dropping row:", index, "Length:", len(test))
    i += 1




Processing i: 0 content/downloaded_posters/Drumline.jpg
Processing i: 100 content/downloaded_posters/Brighton Beach Memoirs.jpg
Processing i: 200 content/downloaded_posters/Bikini Beach.jpg
Processing i: 300 content/downloaded_posters/Ma tou da jue dou.jpg
Processing i: 400 content/downloaded_posters/Catfish.jpg
Processing i: 500 content/downloaded_posters/Mr. Denning Drives North.jpg
Processing i: 600 content/downloaded_posters/Anno zero - guerra nello spazio.jpg
Processing i: 700 content/downloaded_posters/L'arcano incantatore.jpg
Processing i: 800 content/downloaded_posters/Switching Goals.jpg
Processing i: 900 content/downloaded_posters/Scooby-Doo! and the Loch Ness Monster.jpg
Processing i: 1000 content/downloaded_posters/The Exorcism of Emily Rose.jpg
Processing i: 1100 content/downloaded_posters/The Secret of Dr. Kildare.jpg
Processing i: 1200 content/downloaded_posters/The Dawn Patrol.jpg
Processing i: 1300 content/downloaded_posters/Gemma Bovery.jpg
Processing i: 1400 content/

In [18]:
test_np_imgs = np.array(test_imgs)
test.to_csv('content/movie_prediction/test_data.csv')

In [20]:
import pickle
with open('content/movie_prediction/test_np_imgs_norm','wb') as f: pickle.dump(test_np_imgs, f)


train_imgs = []
i = 0
for index, row in train.iterrows():
    if i%100 == 0:
        print("Processing i:", i, str(row['poster_paths']))
    img = resize_img(str(row['poster_paths']))
    if img is not None:
        train_imgs.append(img)
    else:
        train = train.drop(index)
        print("Dropping row:", index, "Length:", len(train))
    i += 1

Processing i: 0 content/downloaded_posters/The Ballad of Nessie.jpg
Processing i: 100 content/downloaded_posters/Link.jpg
Processing i: 200 content/downloaded_posters/Anna, quel particolare piacere.jpg
Processing i: 300 content/downloaded_posters/Blood in the Mobile.jpg
Processing i: 400 content/downloaded_posters/Socha Na tha.jpg
Processing i: 500 content/downloaded_posters/Equinox.jpg
Processing i: 600 content/downloaded_posters/Margot at the Wedding.jpg
Processing i: 700 content/downloaded_posters/Whiteout.jpg
Processing i: 800 content/downloaded_posters/Skyjacked.jpg
Processing i: 900 content/downloaded_posters/Interior. Leather Bar..jpg
Processing i: 1000 content/downloaded_posters/Draegerman Courage.jpg
Processing i: 1100 content/downloaded_posters/Coffin Rock.jpg
Processing i: 1200 content/downloaded_posters/The Keep.jpg
Processing i: 1300 content/downloaded_posters/Ernesto.jpg
Processing i: 1400 content/downloaded_posters/Night Moves.jpg
Processing i: 1500 content/downloaded_po

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 9158 Length: 24858
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 6482 Length: 24857
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 27488 Length: 24856
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 31705 Length: 24855
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 22281 Length: 24854
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgpr

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 16658 Length: 24814
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 2176 Length: 24813
Processing i: 5400 content/downloaded_posters/La vida precoz y breve de Sabina Rivas.jpg
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 22270 Length: 24812
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 19500 Length: 24811
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Droppi

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 32801 Length: 24769
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 17114 Length: 24768
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 22392 Length: 24767
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 5513 Length: 24766
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 18721 Length: 24765
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgp

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 4144 Length: 24721
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 34000 Length: 24720
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 3530 Length: 24719
Processing i: 6100 content/downloaded_posters/Shadow People.jpg
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 34139 Length: 24718
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 27796 Length: 2471

...

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 15088 Length: 22363
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 29054 Length: 22362
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 2282 Length: 22361
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 21207 Length: 22360
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 13981 Length: 22359
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgp

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 28077 Length: 22317
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 8818 Length: 22316
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 18086 Length: 22315
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 17990 Length: 22314
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 17518 Length: 22313
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgp

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 34713 Length: 22270
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 29399 Length: 22269
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 18223 Length: 22268
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 3989 Length: 22267
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 30705 Length: 22266
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgp

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 12192 Length: 22223
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 27220 Length: 22222
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 21495 Length: 22221
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 10050 Length: 22220
Processing i: 24400 content/downloaded_posters/Le Plaisir.jpg
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 33328 Length: 2221

OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 27990 Length: 22178
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 18224 Length: 22177
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 12790 Length: 22176
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 33929 Length: 22175
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Dropping row: 12861 Length: 22174
OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/img

In [21]:
assert len(train) == len(train_imgs)
train_np_imgs = np.array(train_imgs)

train.to_csv("content/movie_prediction/train_data.csv")

import pickle
with open('content/movie_prediction/train_np_imgs_norm','wb') as f: pickle.dump(train_np_imgs, f)